In [67]:
from tinydb import TinyDB, Query
import pandas as pd
from tqdm import tqdm
import datetime



def update_ntd_list(exchange):
    
    #Get all contracts
    contracts_db = TinyDB('/Users/martin/Documents/devel/screener/data_management/contracts_db.json')
    my_query = Query()
    all_instruments = contracts_db.search(my_query.Exchange == exchange)
    contracts_db.close()
    
    # Build initial list of dates
    start = datetime.datetime(2008, 1, 1)
    end = datetime.datetime(2019, 12, 31)
    dt_index = pd.bdate_range(start, end)
    init_vals = dt_index.strftime('%Y-%m-%d').values
    all_days = pd.Series(init_vals)
    
    # Append dates of all existing candlesticks to dates list
    for instrument in tqdm(all_instruments[:10]):
        if not instrument['Status'].startswith('data_ends'):
            continue
        filename = ('/Users/martin/Google Drive/data/screener/' + instrument['Symbol'] + '_' + exchange + '.csv')
        df_histo = pd.read_csv(filename)
        all_days = all_days.append(df_histo['date'])
        
    # Filter dates where no instrument was traded
    counts = all_days.value_counts()
    counts = counts.sort_index()
    while counts.iloc[0] == 1: # Drop trailing no-data dates
        counts = counts.drop(counts.index[0])
    while counts.iloc[-1] == 1: # Drop leading no-data dates
        counts = counts.drop(counts.index[-1])
    counts = counts.drop(counts[counts!=1].index)

    return counts

update_ntd_list('FWB')


100%|██████████| 10/10 [00:00<00:00, 347.88it/s]


2011-04-22    1
2011-04-25    1
2011-12-26    1
2012-04-06    1
2012-04-09    1
             ..
2019-04-19    1
2019-04-22    1
2019-05-01    1
2019-06-10    1
2019-10-03    1
Length: 66, dtype: int64